TO DO:
 - add simple mean evaluation back in
 - add datasets with interactions

# Multi-model comparison (initially on toy data)
 - Data is provided/generated in sci-kit learn friendly format (not including constant?)
 - Implements a train/test split or similar
 - Tries prediction using a list of models that is provided
 - Outputs metrics
  - RMSE, MAE, training and prediction time

# Imports

In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import os
import time

import graphviz
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pygam import LinearGAM, PoissonGAM, s, f
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse
from xgboost import XGBRegressor

%aimport model_wrappers, toy_data_generation, param_dict_generation
from model_wrappers import try_scikit_model, try_statsmodels_model
from toy_data_generation import generate_linear_data, generate_poisson_data
from param_dict_generation import generate_ols_list, generate_glm_list, generate_mlp_list, generate_xgb_list

%matplotlib inline

In [3]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)

# Data generation

In [4]:
datasets = []
for nrows in [10000, 30000]:
    for nvars in [30, 100]:
#         datasets.append((*generate_linear_data(nrows, nvars), f'linear_{nvars}vars_{nrows}rows'))
        datasets.append((*generate_poisson_data(nrows, nvars,
                                                binary_fraction=0.5,
                                                binary_imbalance=10),
                         f'poisson_{nvars}vars_{nrows}rows'))

# Experiment lists

## Statsmodels list

In [5]:
ols_list = generate_ols_list(alpha_list=[1,2,3], L1_wt_list=[0,0.5,1])

glm_list = generate_glm_list('Poisson', alpha_list=[0.001,0.01,0.1])

statsmodels_param_list = ols_list + glm_list

## Scikit list

In [6]:
mlp_list = generate_mlp_list(hidden_layout_list=[(50,), (20, 20)])

xgb_list = generate_xgb_list(depth_list = [2,4,6],
                             learning_rate_list = [0.01, 0.3],
                             objective='Poisson',
                             early_stopping_rounds=5)

scikit_param_list = mlp_list + xgb_list

# Minimal Interesting Product

In [7]:
def mip(X_train, X_test, y_train, y_test, statsmodels_list=None, scikit_list=None):
    results = {}
    for trial in statsmodels_list:
        try:
            results[f"{trial['model_name']}"] = try_statsmodels_model(X_train, X_test, y_train, y_test, trial)
        except:
            print(f'An error occured with {trial["model_name"]}')
    for trial in scikit_list:
        results[f"{trial['model_name']}"] = try_scikit_model(X_train, X_test, y_train, y_test, trial)
    return results

# Loop over datasets

In [8]:
results = {}

for X, y, dataset_name in datasets:
    print(f'\n\n{dataset_name}\n')
    # update scikit_param_list for xgboost early stopping eval_sets if present
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=303)
    model_results = mip(
        X_train, X_test, y_train, y_test,
        statsmodels_list=statsmodels_param_list, scikit_list=scikit_param_list
    )
    for key in model_results:
        results[(dataset_name, key)] = model_results[key]
    
display(pd.DataFrame(results).T)



poisson_30vars_10000rows

OLS_no_reg trained in 0.03 seconds
OLS_alpha_1_L1_0 trained in 0.02 seconds
OLS_alpha_1_L1_0.5 trained in 0.03 seconds
OLS_alpha_1_L1_1 trained in 0.03 seconds
OLS_alpha_2_L1_0 trained in 0.02 seconds
OLS_alpha_2_L1_0.5 trained in 0.03 seconds
OLS_alpha_2_L1_1 trained in 0.03 seconds
OLS_alpha_3_L1_0 trained in 0.02 seconds
OLS_alpha_3_L1_0.5 trained in 0.04 seconds
OLS_alpha_3_L1_1 trained in 0.07 seconds
GLM_Poisson_no_reg trained in 0.16 seconds
GLM_Poisson_alpha_0.001 trained in 0.77 seconds
GLM_Poisson_alpha_0.01 trained in 0.26 seconds
GLM_Poisson_alpha_0.1 trained in 0.23 seconds
MLP_50 trained in 0.51 seconds
MLP_20_20 trained in 0.41 seconds
XGB_Poisson_maxd_2_lr_0.01 trained in 0.62 seconds
XGB_Poisson_maxd_2_lr_0.3 trained in 0.22 seconds
XGB_Poisson_maxd_4_lr_0.01 trained in 1.05 seconds
XGB_Poisson_maxd_4_lr_0.3 trained in 0.27 seconds
XGB_Poisson_maxd_6_lr_0.01 trained in 1.33 seconds
XGB_Poisson_maxd_6_lr_0.3 trained in 0.39 seconds


poisson_

/home/arthurjackrussell/repos/customer-deliverables/bm-venv/lib/python3.6/site-packages/statsmodels/genmod/families/links.py:477: RuntimeWarning: overflow encountered in exp
  return np.exp(z)
/home/arthurjackrussell/repos/customer-deliverables/bm-venv/lib/python3.6/site-packages/statsmodels/genmod/families/family.py:131: RuntimeWarning: invalid value encountered in multiply
  return 1. / (self.link.deriv(mu)**2 * self.variance(mu))
/home/arthurjackrussell/repos/customer-deliverables/bm-venv/lib/python3.6/site-packages/statsmodels/genmod/generalized_linear_model.py:1129: RuntimeWarning: invalid value encountered in multiply
  - self._offset_exposure)


An error occured with GLM_Poisson_no_reg
GLM_Poisson_alpha_0.001 trained in 2.84 seconds
GLM_Poisson_alpha_0.01 trained in 0.86 seconds
GLM_Poisson_alpha_0.1 trained in 0.76 seconds
MLP_50 trained in 1.23 seconds
MLP_20_20 trained in 0.53 seconds
XGB_Poisson_maxd_2_lr_0.01 trained in 1.94 seconds
XGB_Poisson_maxd_2_lr_0.3 trained in 0.50 seconds
XGB_Poisson_maxd_4_lr_0.01 trained in 3.46 seconds
XGB_Poisson_maxd_4_lr_0.3 trained in 0.84 seconds
XGB_Poisson_maxd_6_lr_0.01 trained in 5.00 seconds
XGB_Poisson_maxd_6_lr_0.3 trained in 1.08 seconds


poisson_30vars_30000rows

OLS_no_reg trained in 0.07 seconds
OLS_alpha_1_L1_0 trained in 0.07 seconds
OLS_alpha_1_L1_0.5 trained in 0.07 seconds
OLS_alpha_1_L1_1 trained in 0.06 seconds
OLS_alpha_2_L1_0 trained in 0.05 seconds
OLS_alpha_2_L1_0.5 trained in 0.05 seconds
OLS_alpha_2_L1_1 trained in 0.06 seconds
OLS_alpha_3_L1_0 trained in 0.06 seconds
OLS_alpha_3_L1_0.5 trained in 0.05 seconds
OLS_alpha_3_L1_1 trained in 0.05 seconds
GLM_Poisson_

mse_test  \
poisson_30vars_10000rows  OLS_no_reg                  0.010559   
                          OLS_alpha_1_L1_0            0.010562   
                          OLS_alpha_1_L1_0.5          0.010667   
                          OLS_alpha_1_L1_1            0.010667   
                          OLS_alpha_2_L1_0            0.010588   
                          OLS_alpha_2_L1_0.5          0.010667   
                          OLS_alpha_2_L1_1            0.010667   
                          OLS_alpha_3_L1_0            0.010604   
                          OLS_alpha_3_L1_0.5          0.010667   
                          OLS_alpha_3_L1_1            0.010667   
                          GLM_Poisson_no_reg          0.010553   
                          GLM_Poisson_alpha_0.001     0.010549   
                          GLM_Poisson_alpha_0.01      0.010669   
                          GLM_Poisson_alpha_0.1       0.020706   
                          MLP_50                      0.012207   
                          MLP_20_20                   0.011976   
                          XGB_Poisson_maxd_2_lr_0.01  0.043233   
                          XGB_Poisson_maxd_2_lr_0.3   0.010482   
                          XGB_Poisson_maxd_4_lr_0.01  0.043243   
                          XGB_Poisson_maxd_4_lr_0.3   0.010525   
                          XGB_Poisson_maxd_6_lr_0.01  0.043243   
                          XGB_Poisson_maxd_6_lr_0.3   0.010646   
poisson_100vars_10000rows OLS_no_reg                  0.012696   
                          OLS_alpha_1_L1_0            0.012492   
                          OLS_alpha_1_L1_0.5          0.012667   
                          OLS_alpha_1_L1_1            0.012667   
                          OLS_alpha_2_L1_0            0.012526   
                          OLS_alpha_2_L1_0.5          0.012667   
                          OLS_alpha_2_L1_1            0.012667   
                          OLS_alpha_3_L1_0            0.012549   
                          OLS_alpha_3_L1_0.5          0.012667   
                          OLS_alpha_3_L1_1            0.012667   
                          GLM_Poisson_alpha_0.001     0.012522   
                          GLM_Poisson_alpha_0.01      0.012631   
                          GLM_Poisson_alpha_0.1       0.022746   
                          MLP_50                      0.018979   
                          MLP_20_20                   0.013929   
                          XGB_Poisson_maxd_2_lr_0.01  0.045029   
                          XGB_Poisson_maxd_2_lr_0.3   0.012537   
                          XGB_Poisson_maxd_4_lr_0.01  0.045206   
                          XGB_Poisson_maxd_4_lr_0.3   0.012630   
                          XGB_Poisson_maxd_6_lr_0.01  0.045363   
                          XGB_Poisson_maxd_6_lr_0.3   0.012554   
poisson_30vars_30000rows  OLS_no_reg                  0.010221   
                          OLS_alpha_1_L1_0            0.010234   
                          OLS_alpha_1_L1_0.5          0.010333   
                          OLS_alpha_1_L1_1            0.010333   
                          OLS_alpha_2_L1_0            0.010255   
                          OLS_alpha_2_L1_0.5          0.010333   
                          OLS_alpha_2_L1_1            0.010333   
                          OLS_alpha_3_L1_0            0.010269   
                          OLS_alpha_3_L1_0.5          0.010333   
                          OLS_alpha_3_L1_1            0.010333   
                          GLM_Poisson_no_reg          0.010232   
                          GLM_Poisson_alpha_0.001     0.010231   
                          GLM_Poisson_alpha_0.01      0.010358   
                          GLM_Poisson_alpha_0.1       0.020487   
                          MLP_50                      0.011322   
                          MLP_20_20                   0.010694   
                          XGB_Poisson_maxd_2_lr_0.01  0.043006   
                          XG